In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the path to the .zip file and the destination folder
zip_file_path = '/content/drive/MyDrive/model.zip'  # Replace with your file path
destination_folder = '/content/new'  # Replace with your desired folder path

# Make sure the destination directory exists
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print("Files extracted to:", destination_folder)

Files extracted to: /content/new


In [ ]:
!pip install keras_preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
from tensorflow import keras
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense , Conv2D , Dropout , Flatten , MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
TRAIN_DIR = '/content/new/training/train'
TEST_DIR = '/content/new/training/test'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

rus_internalpassport completed
est_id completed
fin_id completed
srb_passport completed
aze_passport completed
lva_passport completed
svk_id completed
grc_passport completed
esp_id completed
alb_id completed


In [ ]:
print(train)

                                                  image                 label
0     /content/new/training/train/rus_internalpasspo...  rus_internalpassport
1     /content/new/training/train/rus_internalpasspo...  rus_internalpassport
2     /content/new/training/train/rus_internalpasspo...  rus_internalpassport
3     /content/new/training/train/rus_internalpasspo...  rus_internalpassport
4     /content/new/training/train/rus_internalpasspo...  rus_internalpassport
...                                                 ...                   ...
5030        /content/new/training/train/alb_id/a144.jpg                alb_id
5031        /content/new/training/train/alb_id/a504.jpg                alb_id
5032        /content/new/training/train/alb_id/a221.jpg                alb_id
5033        /content/new/training/train/alb_id/a105.jpg                alb_id
5034        /content/new/training/train/alb_id/a302.jpg                alb_id

[5035 rows x 2 columns]


In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

rus_internalpassport completed
est_id completed
fin_id completed
srb_passport completed
aze_passport completed
lva_passport completed
svk_id completed
grc_passport completed
esp_id completed
alb_id completed


In [ ]:
print(test)

                                                  image                 label
0     /content/new/training/test/rus_internalpasspor...  rus_internalpassport
1     /content/new/training/test/rus_internalpasspor...  rus_internalpassport
2     /content/new/training/test/rus_internalpasspor...  rus_internalpassport
3     /content/new/training/test/rus_internalpasspor...  rus_internalpassport
4     /content/new/training/test/rus_internalpasspor...  rus_internalpassport
...                                                 ...                   ...
1995          /content/new/training/test/alb_id/a37.jpg                alb_id
1996         /content/new/training/test/alb_id/a177.jpg                alb_id
1997          /content/new/training/test/alb_id/a36.jpg                alb_id
1998         /content/new/training/test/alb_id/a144.jpg                alb_id
1999         /content/new/training/test/alb_id/a105.jpg                alb_id

[2000 rows x 2 columns]


In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale")
        img = np.array(img)
        features.append(img)
    features =np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/5035 [00:00<?, ?it/s]

In [ ]:
test_features = extract_features(test['image'])

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [ ]:
y_train = le.transform(train['label'])

In [ ]:
y_test = le.transform(test['label'])

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Initialize and fit LabelEncoder only once
le = LabelEncoder()
y_train = le.fit_transform(train['label'])  # Fit and transform on training labels only
y_test = le.transform(test['label'])        # Transform test labels with the same encoder

# Convert labels to categorical (one-hot encoding) with 10 classes
num_classes = 10
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Ensure x_train and x_test are correctly shaped and normalized
x_train = x_train.reshape(-1, 48, 48, 1).astype("float32") / 255.0  # Normalize images
x_test = x_test.reshape(-1, 48, 48, 1).astype("float32") / 255.0    # Normalize images

# Model creation
model = Sequential()

# Add Input and Convolutional layers
model.add(Input(shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with an optimizer and loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [ ]:
# Fit the model using the data generator
train_generator = datagen.flow(x_train, y_train, batch_size=32)

# Train the model
model.fit(train_generator, epochs=30, validation_data=(x_test, y_test))  # x_test is already normalized

# Save the model
model.save("ID_detector_improved.h5")

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


158/158 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.0972 - loss: 2.3040 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 2/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.1083 - loss: 2.3023 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 3/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.1012 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3029
Epoch 4/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.1046 - loss: 2.3025 - val_accuracy: 0.1000 - val_loss: 2.3028
Epoch 5/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.1063 - loss: 2.3025 - val_accuracy: 0.1000 - val_loss: 2.3028
Epoch 6/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.1030 - loss: 2.3029 - val_accuracy: 0.1000 - val_loss: 2.3028
Epoch 7/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.1121 - loss: 2.3023 - val_accuracy: 0.1000 - val_loss: 2.3028
Epoch 8/30
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.1101 - loss: 2.3022 - val_accuracy: 0.10